#Authoring Agents in Code

If you need the additional flexibility and control over your agents, we recommend developing agents in code. At Databricks, we are a framework agnostic platform so you can bring any framework to orchestrate against your LLM. 

For this notebook, we will be using DSPy to quickly define tools and deploy agent endpoints. 

First, we will define the tools we want our Agent to use. Then, we will use DSPy to create a signature that DSPy will compile into a Prompt to send to the LLM. We will test this Agent, then deploy on Databricks. 

In [0]:
%pip install --upgrade dspy==3.0.0b3 mlflow unitycatalog-ai[databricks] databricks-sdk databricks-vectorsearch databricks-agents git+https://github.com/BerriAI/litellm.git@main
dbutils.library.restartPython()

#Step 1: Define your Tools

As you saw in the past notebooks, it's critical that we provide tools to our agents so that they become data intelligent and can access information they may not have access to. 

Let's use the Vector Search Endpoint created by the Knowledge Assistant Agent Brick 

In [0]:
endpoint_name= "ka-01481fed-endpoint" #change this to your agent bricks endpoint

dbutils.widgets.text("endpoint_name", endpoint_name)

In [0]:
import mlflow
import mlflow.deployments
import pandas as pd
import time

def sec_search(sec_question):
    """This function takes a user's question and returns relevant information from companies' 10-K, 8-K, and annual reports."""
    client = mlflow.deployments.get_deploy_client("databricks")
    response = client.predict(
        endpoint=dbutils.widgets.get("endpoint_name"),
        inputs={"dataframe_split": {
            "columns": ["input"],
            "data": [[
                [{"role": "user", "content": sec_question}]
            ]]
        }}
    )
    return response['predictions']['output'][0]['content'][0]['text']

In [0]:
result = sec_search("Who is Michael (Mike) Mohan?")
print(result)

To keep the workshop simple, we will only use one tool and keep this as a simple RAG agent. 

#Step 2: Define the DSPy Signature 

Now we need to define the DSPy signature. This signature is explain what we are trying to accomplish and define the inputs and outputs for the LLM. 

In [0]:
import dspy
import mlflow
llm = dspy.LM('databricks/databricks-llama-4-maverick', cache=False)
# llm = dspy.LM('databricks/databricks-meta-llama-3-1-8b-instruct', cache=False)
# llm = dspy.LM('databricks/databricks-claude-3-7-sonnet', cache=False)
dspy.configure(lm=llm)
mlflow.dspy.autolog()
# mlflow.set_experiment(experiment_id="835bf9ec05f24eb09289e8030853d968")

In [0]:
import dspy
from typing import Literal
class rag_signature(dspy.Signature):
  """
  use the given tools to answer the question
  """ 
  question: str = dspy.InputField()
  response: str = dspy.OutputField() 

#Step 3: Test the Signature

Let's see if we can get the LLM to execute the tool

In [0]:
rag = dspy.ReAct(rag_signature, tools=[sec_search], max_iters=1)

In [0]:
question = "Who is Michael (Mike) Mohan?"
result = rag(question=question)
print(result)

In [0]:
print(f"The response: {result.response}\n")

#Call out: MLflow Traces

Mlflow traces gives us additional visibility into what is happening under the hood. As you develop more complex Agents, it will be difficult to see what is happening or what decisions the LLM makes. Sometimes, your agent will finish executing even if the tool fails to complete but you will have no idea this happened. It's important to add tracing to you agent development workflow or else you will be working in the dark. 


#Let's Deploy this! 

At Databricks, we deploy our agents in code using the MLflow ChatAgent object. This allows us to use agents.deploy that creates an agent endpoint complete with a feedback model and review app for feedback from our Subject Matter Experts

In [0]:
%%writefile agent.py

from typing import Any, Generator, Optional
import mlflow
import mlflow.deployments
from databricks.sdk import WorkspaceClient
from mlflow.entities import SpanType
from mlflow.pyfunc.model import ChatAgent
from mlflow.types.agent import (
    ChatAgentMessage,
    ChatAgentResponse,
    ChatContext,
)
import dspy
import uuid
import os


mlflow.dspy.autolog()
mlflow.set_experiment(experiment_id="835bf9ec05f24eb09289e8030853d968")
LLM_ENDPOINT_NAME = "databricks-llama-4-maverick"
lm = dspy.LM(model=f"databricks/{LLM_ENDPOINT_NAME}")
dspy.settings.configure(lm=lm)
endpoint_name= "ka-ca0f678f-endpoint" #update this

class rag_signature(dspy.Signature):
  """
  use the given tools to answer the question
  """ 
  question: str = dspy.InputField()
  response: str = dspy.OutputField() 

class DSPyChatAgent(ChatAgent):     
    def __init__(self):
      self.rag_signature = rag_signature
      self.endpoint_name = endpoint_name
      self.rag_agent = dspy.ReAct(self.rag_signature, tools=[self.sec_search],max_iters=1)
    

    def sec_search(self, databricks_question):
        """This function needs the User's question. The question is used to pull documentation about Databricks. Use the information to answer the user's question"""
        client = mlflow.deployments.get_deploy_client("databricks")
        response = client.predict(
            endpoint=self.endpoint_name,
            inputs={"dataframe_split": {
                "columns": ["input"],
                "data": [[
                    [{"role": "user", "content": databricks_question}]
                ]]
            }}
        )
        return response['predictions']['output'][0]['content'][0]['text']
      
    def prepare_message_history(self, messages: list[ChatAgentMessage]):
        history_entries = []
        # Assume the last message in the input is the most recent user question.
        for i in range(0, len(messages) - 1, 2):
            history_entries.append({"question": messages[i].content, "answer": messages[i + 1].content})
        return dspy.History(messages=history_entries)
      
    @mlflow.trace(span_type=SpanType.AGENT)
    def predict(
        self,
        messages: list[ChatAgentMessage],
        context: Optional[ChatContext] = None,
        custom_inputs: Optional[dict[str, Any]] = None,
    ) -> ChatAgentResponse:
        latest_question = messages[-1].content
        response = self.rag_agent(question=latest_question).response
        return ChatAgentResponse(
            messages=[ChatAgentMessage(role="assistant", content=response, id=uuid.uuid4().hex)]
        )

from mlflow.models import set_model
AGENT = DSPyChatAgent()
set_model(AGENT)

In [0]:
dbutils.library.restartPython()

In [0]:
from agent import AGENT

AGENT.predict({"messages": [{"role": "user", "content": "What were the key financial highlights for Amazon in 2019?"}]})

#Let's evaluate our agent

One of the most powerful parts of MLflow 3.0 is the evaluation frameworks. You can track all your evaluation runs and compare them in the MLflow Experiments page. All activity such as traces, judges and more can be found in that experiment. 

You can work with your agents.py file to update the agent if you find it not performing to your expectations

In [0]:
import mlflow
from mlflow.genai import datasets, evaluate, scorers
from agent import AGENT
from mlflow.genai.scorers import (
    RetrievalGroundedness,
    RelevanceToQuery,
    Safety,
    Guidelines,
)

# You can manually set an experiment to organize your traces and evaluation runs under one experiment. However, mlflow.autolog will automatically create one for you to use associated with this notebook. 

# mlflow.set_experiment(experiment_id="835bf9ec05f24eb09289e8030853d968")

#A dummy question set to run our evaluations against
eval_dataset = [
    {
        "inputs": {"question": "What were the key financial highlights for Amazon in 2019?"},
    },
    {
        "inputs": {"question": "Who was the CEO of Nike in 2018?"},
    },
    {
        "inputs": {"question": "What strategic initiatives did Adobe announce in their 2018 reports?"},
    },
    {
        "inputs": {"question": "What were Walmart's sustainability goals mentioned in their 2020 documents?"},
    },
]

#You can set your own judges or scorers here! Guidelines are a quick way to prompt how you want your LLM as a judge to evaluate the responses.
bricks_scorers = [
  Guidelines(
    name="relevant_content",
    guidelines="The generated response is similar to the retrieved information",
  ),
  Guidelines(
    name="professional",
    guidelines="the generated response sounds professional and polite",
  ) ,
  Guidelines(
    name="concise",
    guidelines="the generated response is concise and to the point",
  ),
  Guidelines(
    name="no_personal_opinion",
    guidelines="the generated response does not contain any personal opinion or bias",
  ),
  RelevanceToQuery(),
  Safety()
]

def predict(question):
  result = AGENT.predict({"messages": [{"role": "user", "content": question}]})
  return {"response": result.messages[0].content}

# Step 3: Run evaluation
evaluate(
  data=eval_dataset,
  predict_fn=predict,
  scorers=bricks_scorers
)

# Results will appear back in this UI, you can click the button below to review the evaluation results

In [0]:
import mlflow
from agent import LLM_ENDPOINT_NAME
from mlflow.models.resources import (
    DatabricksVectorSearchIndex,
    DatabricksServingEndpoint,
)
from pkg_resources import get_distribution

resources = [
    DatabricksServingEndpoint(endpoint_name="databricks-llama-4-maverick"),
    DatabricksServingEndpoint(endpoint_name=dbutils.widgets.get("endpoint_name")),
]

with mlflow.start_run():
    logged_agent_info = mlflow.pyfunc.log_model(
        name="agent",
        python_model="agent.py",
        # input_example=input_example,
        extra_pip_requirements=[f"databricks-connect=={get_distribution('databricks-connect').version}"],
        resources=resources,
    )

In [0]:
mlflow.set_registry_uri("databricks-uc")
email = dbutils.notebook.entry_point.getDbutils().notebook().getContext().userName().get()
email = email.split("@")[0].replace(".","_")

catalog = "genai_in_production_demo_catalog"
schema = "agents"
model_name = f"ai_pioneer_agent_{email}"
UC_MODEL_NAME = f"{catalog}.{schema}.{model_name}"
print(f"Your model name: {UC_MODEL_NAME}")
# register the model to UC
uc_registered_model_info = mlflow.register_model(model_uri=logged_agent_info.model_uri, name=UC_MODEL_NAME)

#Now let's deploy it!

###*NOTE* 
Unfortunately, due to the limitations of the trial workspace, we are not able to deploy more than one agent endpoint. So please refer to the presenters to demonstrate the agent endpoint and review app

In [0]:
from databricks import agents
import os

agents.deploy(UC_MODEL_NAME, uc_registered_model_info.version, environment_vars={"MLFLOW_TRACKING_URI": "databricks"})

#Congrats! You have an Agent! 

Check out the links above to see your agents deploy!



#But we aren't done! 

Make sure to Evaluate your agent before and after! Having good evaluations grounded in your metrics to monitor for any diverging behavior 

In [0]:
import mlflow
from mlflow.genai import evaluate, scorers
import mlflow.genai.datasets
from agent import AGENT
from mlflow.genai.scorers import (
    RetrievalGroundedness,
    RelevanceToQuery,
    Safety,
    Guidelines,
)

# mlflow.set_experiment(experiment_id="835bf9ec05f24eb09289e8030853d968") #use this if you want to use more traces or preloaded traces
traces = mlflow.search_traces(
  max_results=5,
  order_by=["timestamp DESC"]
)

bricks_scorers = [
  Guidelines(
    name="relevant_content",
    guidelines="The generated response is relevant to the retrieved information",
  ),
  Guidelines(
    name="professional",
    guidelines="the generated response sounds professional and polite",
  ) ,
  Guidelines(
    name="concise",
    guidelines="the generated response is concise and to the point",
  ),
  Guidelines(
    name="no_personal_opinion",
    guidelines="the generated response does not contain any personal opinion or bias",
  ),
  RelevanceToQuery(),
  Safety()
]

# Step 3: Run evaluation
evaluate(
  data=traces,
  scorers=bricks_scorers
)

# Results will appear back in this UI